In [1]:
from pathlib import Path
import pandas as pd

from stables.core.sources import LlamaStableAPI
from stables.core.transformers import LlamaStableTransformer

root_path = Path.cwd().parent

In [2]:
api = LlamaStableAPI()
raw_data = await api.get_circulating()

transformer = LlamaStableTransformer()

In [3]:
stable_df = transformer.circulating_to_stable_df(raw_data)
chain_circulating_df = transformer.circulating_to_chain_circulating_df(raw_data)

In [4]:
chain_circulating_df = chain_circulating_df.merge(stable_df, on='id', how='left')
chain_circulating_df

,id,chain,circulating,circulatingPrevDay,circulatingPrevWeek,circulatingPrevMonth,name,symbol,gecko_id,peg_type,peg_mechanism,total_circulating
0,1,Optimism,2.256995e+08,2.718583e+08,3.658341e+08,3.925224e+08,Tether,USDT,tether,peggedUSD,fiat-backed,1.457962e+11
1,1,Statemine,2.399930e+05,2.399930e+05,2.399930e+05,2.399930e+05,Tether,USDT,tether,peggedUSD,fiat-backed,1.457962e+11
2,1,Manta,6.084238e+06,6.104238e+06,6.119730e+06,6.181048e+06,Tether,USDT,tether,peggedUSD,fiat-backed,1.457962e+11
3,1,Harmony,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Tether,USDT,tether,peggedUSD,fiat-backed,1.457962e+11
4,1,ThunderCore,6.313365e+05,6.180446e+05,6.162378e+05,6.187428e+05,Tether,USDT,tether,peggedUSD,fiat-backed,1.457962e+11
...,...,...,...,...,...,...,...,...,...,...,...,...
970,257,Ethereum,1.479654e+08,1.480109e+08,0.000000e+00,0.000000e+00,OpenEden TBILL,TBILL,openeden-tbill,peggedUSD,fiat-backed,1.683923e+08
971,257,Solana,1.053696e+07,1.056852e+07,0.000000e+00,0.000000e+00,OpenEden TBILL,TBILL,openeden-tbill,peggedUSD,fiat-backed,1.683923e+08
972,257,Arbitrum,5.226682e+06,5.226682e+06,0.000000e+00,0.000000e+00,OpenEden TBILL,TBILL,openeden-tbill,peggedUSD,fiat-backed,1.683923e+08
973,257,Ripple,4.663215e+06,4.663215e+06,0.000000e+00,0.000000e+00,OpenEden TBILL,TBILL,openeden-tbill,peggedUSD,fiat-backed,1.683923e+08


In [5]:
chain_circulating_df[
    chain_circulating_df['chain'] == "StarkNet"
]

,id,chain,circulating,circulatingPrevDay,circulatingPrevWeek,circulatingPrevMonth,name,symbol,gecko_id,peg_type,peg_mechanism,total_circulating
14,1,StarkNet,1.262543e+07,1.330685e+07,1.350584e+07,1.303694e+07,Tether,USDT,tether,peggedUSD,fiat-backed,1.457962e+11
96,2,StarkNet,4.615795e+07,4.535382e+07,4.553565e+07,4.985253e+07,USD Coin,USDC,usd-coin,peggedUSD,fiat-backed,6.177568e+10
257,5,StarkNet,4.299680e+05,4.299680e+05,4.299680e+05,4.727080e+05,Dai,DAI,dai,peggedUSD,crypto-backed,4.105683e+09
791,177,StarkNet,2.926095e+04,2.915099e+04,2.912370e+04,2.864260e+04,UNO,UNO,nostra-uno,peggedUSD,fiat-backed,2.926095e+04
840,206,StarkNet,3.152828e+05,3.153242e+05,3.165201e+05,3.167767e+05,Opus CASH,CASH,opus-cash,peggedUSD,crypto-backed,3.152828e+05


In [6]:
selected_cols = [
    'date', 'circulating', 
    # 'bridgedTo', 
        'minted', 
    #  'unreleased', 
        'chain'
                 ]
df = pd.DataFrame()
for stable_id in stable_df['id']:
    _df = pd.read_parquet(root_path / 'data/historical' / f'2025-04-23_{stable_id}.parquet')[selected_cols]
    _df['id'] = stable_id
    df = pd.concat([df, _df])
df['date'] = pd.to_datetime(df['date'], unit='s')


In [7]:
df[
     df['chain'] == "StarkNet"
]

,date,circulating,minted,chain,id
0,2024-02-18,7.356377e+06,NaN,StarkNet,1
1,2024-02-19,7.356760e+06,NaN,StarkNet,1
2,2024-02-20,7.566100e+06,NaN,StarkNet,1
3,2024-02-21,1.087976e+07,NaN,StarkNet,1
4,2024-02-22,1.102499e+07,NaN,StarkNet,1
...,...,...,...,...,...
167,2025-04-19,3.166990e+05,316699.000000,StarkNet,206
168,2025-04-20,3.166790e+05,316679.000000,StarkNet,206
169,2025-04-21,3.153440e+05,315344.000000,StarkNet,206
170,2025-04-22,3.153240e+05,315324.000000,StarkNet,206
